In [19]:
#import the necessary packages
import torch
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from torch.nn import BCEWithLogitsLoss
from torchvision import transforms
from torch.utils.data import DataLoader
# base path of the dataset
from pyimagesearch.model import UNet
DATASET_PATH = "/data/DeepSense6G/stationary_scenarios/scenario31/generated_dataset"

# define the path to the images and masks dataset
IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "radar")
MASK_DATASET_PATH = os.path.join(DATASET_PATH, "lidar")



In [4]:
# define the test split
TEST_SPLIT = 0.15

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3
# initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 64
# define the input image dimensions
INPUT_IMAGE_WIDTH = 256
INPUT_IMAGE_HEIGHT = 256
# define threshold to filter weak predictions
THRESHOLD = 0.5
# define the path to the base output directory
BASE_OUTPUT = "examples/PyTorch_tutorial/output"
# define the path to the output serialized model, model training
# plot, and testing image paths
# MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_tgs_salt.pth")
# PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
# TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

In [5]:
# import the necessary packages
from torch.utils.data import Dataset
import cv2
import numpy as np

class SegmentationDataset(Dataset):
	def __init__(self, imagePaths, maskPaths, transforms):
		# store the image and mask filepaths, and augmentation
		# transforms
		self.imagePaths = imagePaths
		self.maskPaths = maskPaths
		self.transforms = transforms
		
	def __len__(self):
		# return the number of total samples contained in the dataset
		return len(self.imagePaths)
	
	def __getitem__(self, idx):
		# grab the image path from the current index
		imagePath = self.imagePaths[idx]
		maskPath = self.maskPaths[idx]
		# load the image from disk, swap its channels from BGR to RGB,
		# and read the associated mask from disk in grayscale mode
		image = np.load(imagePath)
		
		mask = np.load(maskPath)
		
		# check to see if we are applying any transformations
		if self.transforms is not None:
			# apply the transformations to both image and its mask
			image = self.transforms(image)
			mask = self.transforms(mask)
			
		# return a tuple of the image and its mask
		return (image, mask)


In [6]:
import numpy as np
from os import listdir



def list_files(directory, extension):
    return (os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(extension))


directory_im = IMAGE_DATASET_PATH
directory_mas = MASK_DATASET_PATH
extension = '.npy'


imagePaths = sorted(list_files(directory_im, extension))
maskPaths = sorted(list_files(directory_mas, extension))

split = train_test_split(imagePaths, maskPaths,
	test_size=0.15, random_state=42)

(trainImages, testImages) = split[:2]
(trainMasks, testMasks) = split[2:]

In [39]:
from torchvision import transforms
transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((256,256))])

In [47]:
trainDS = SegmentationDataset(imagePaths=trainImages, maskPaths=trainMasks,
	transforms=transforms)
testDS = SegmentationDataset(imagePaths=testImages, maskPaths=testMasks,
    transforms=transforms)
trainDS[0][0].shape

torch.Size([3, 256, 256])

In [14]:
from pyimagesearch import config
print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")

trainLoader = DataLoader(trainDS, shuffle=True,
	batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
	num_workers=os.cpu_count())
testLoader = DataLoader(testDS, shuffle=False,
	batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
	num_workers=os.cpu_count())

[INFO] found 5960 examples in the training set...
[INFO] found 1052 examples in the test set...


In [25]:
from torch.optim import Adam
import time
from tqdm import tqdm
unet = UNet().to(config.DEVICE)

# initialize loss function and optimizer
lossFunc = BCEWithLogitsLoss()
opt = Adam(unet.parameters(), lr=config.INIT_LR)

# calculate steps per epoch for training and test set
trainSteps = len(trainDS) // config.BATCH_SIZE
testSteps = len(testDS) // config.BATCH_SIZE
H = {"train_loss": [], "test_loss": []}

# loop over epochs
print("[INFO] training the network...")
startTime = time.time()

[INFO] training the network...


In [49]:
unet(x)

RuntimeError: CUDA out of memory. Tried to allocate 190.00 MiB (GPU 0; 7.78 GiB total capacity; 6.02 GiB already allocated; 37.00 MiB free; 6.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [48]:
for e in tqdm(range(config.NUM_EPOCHS)):
	# set the model in training mode
	unet.train()
	
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalTestLoss = 0
	
	# loop over the training set
	for (i, (x, y)) in enumerate(trainLoader):
		
		# send the input to the device
		(x, y) = (x.to(config.DEVICE).float(), y.to(config.DEVICE).float())
		
		# perform a forward pass and calculate the training loss
		pred = unet(x)
		loss = lossFunc(pred, y)
		
		# first, zero out any previously accumulated gradients, then
		# perform backpropagation, and then update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()
		
		# add the loss to the total training loss so far
		totalTrainLoss += loss
		
	# switch off autograd
	with torch.no_grad():
		# set the model in evaluation mode
		unet.eval()
		
		# loop over the validation set
		for (x, y) in testLoader:
			
			# send the input to the device
			(x, y) = (x.to(config.DEVICE), y.to(config.DEVICE))
			
			# make the predictions and calculate the validation loss
			pred = unet(x)
			totalTestLoss += lossFunc(pred, y)
			
	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgTestLoss = totalTestLoss / testSteps
	
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
	
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, config.NUM_EPOCHS))
	print("Train loss: {:.6f}, Test loss: {:.4f}".format(
		avgTrainLoss, avgTestLoss))
	
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))

# plot the training loss
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["test_loss"], label="test_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(config.PLOT_PATH)
# serialize the model to disk
torch.save(unet, config.MODEL_PATH)

  0%|          | 0/40 [00:03<?, ?it/s]


ValueError: Target size (torch.Size([64, 1, 256, 256])) must be the same as input size (torch.Size([64, 1, 128, 128]))

In [11]:
trainLoader = DataLoader(trainDS, shuffle=True,
	batch_size=6, pin_memory=PIN_MEMORY,
	num_workers=os.cpu_count())
testLoader = DataLoader(testDS, shuffle=False,
	batch_size=6, pin_memory=PIN_MEMORY,
	num_workers=os.cpu_count())

In [12]:

dataset = trainLoader.dataset



single_example = dataset

In [ ]:
trainDS[0]

In [ ]:
inputs, classes = next(iter(trainLoader)) 